# Introduction
- このノートブックでは、音から抽出される特徴量について勉強します。
- 日本語の解説はあくまでイメージです。
- 正確な理解のために数式をじっと見てください。

# Features
- CDの音源を例に取ると、1秒間に44100個ものデータが格納されています。つまり、sample rate = 44100 となります。
- このとき、値一つ一つは音の大きさを意味しますが、それ自体から意味を見出すのは難しいことは容易に想像できます。
- 例えば、小さい音が急に大きくなって100という値を取った。ずっと同程度の大きさ音が続いていて、その値が100だったというのは、同じ100という大きさの音でも意味がまったく違います。
- 時系列データというのは、前後の関係が重要になります。
- もちろん、Energy というような大きさそのものから抽出される特徴もありますが、通常は前後の関係から抽出される量を用います。
- 今回は例として、windoe_size = 0.5 s, window_stride = 0.25 s という状況を想定します。
- window_stride と、window_size はそれぞれ短時間フーリエ変換のパラメータです。(調べてください。)
---
**以下でよく使う特徴量について簡単に解説します。**  
- 使う記号について   
sr = sample_rate  
$X$ は、signalをフーリエ変換したものです。ある周波数$f$のときの強度を$X(f)$と書きます。  
$x$ は、signal から window_size 分だけ、抜き出したものです。  
$x = \{x_1, x_2, ... \}$
例) sample rate = 44100 のデータかつ、window_size = 0.5 sec の場合、$x$には$44100 \times 0.5$ だけのデータが入っています。  

問題)  
データが10秒分ある場合、signalのデータ点は、$44100 \times 10$だけデータがあります。  
このとき、抽出される特徴量のデータ数は何個あるでしょうか。  
計算してみてください。

---

## $ZCR(x)$
**ZCR:zero crossing rate**

- x が＋ から - に切り替わるタイミング、つまり、$x=0$を通過する回数です。
- この量が重要な理由は私はよくわかりません。
- 細かい係数などは、ライブラリによって違いますの、ZCRの値そのもは単純に回数ではないことは頭に置いておいてください。

---

## $Energy(x)$
$$
Energy = \sum _i x_i ^2
$$

見てのとおりです。

---

## $EnergyEntropy(x)$
シグナル $x$を10こずつに分け、それぞれで、エネルギーを計算します(微小エネルギーと今は呼びます)。  
微小エネルギーを $e_j$と書きます。 また、全体のエネルギ-を$E$と書けば、エナジーエントロピーは、

$$
EnergyEntropy = - \sum _j \frac{e_j}{E} \log_2 \frac{e_j}{E}
$$

$e_j /E$は0~1の値を取ります。 
微小エネルギーが偏っていれば、大きな値をとり、平均的に散らばっていれば、小さな値をとります。

---

## $SpectralCentroid(X)$
$$
SpectralCentroid = \frac {\int f |X(f)|df}{\int  |X(f)|df}
$$

$p(f)df = \frac {|X(f)|df}{\int  |X(f)|df}$ は、微小区間 $df$ 内の周波数$f$の確率(p(f)は確率密度)を表すことが式からわかります。($f$で積分すると1になりますね。)  
微小区間の確率$p(f)df$ と、$f$ を掛けて積分しているので、$f$の平均値を計算していることがわかると思います。  
スペクトルの平均 = スペクトルの重心 = SpectralCentroid というわけです。


この式は、以下の式と同じ形だと思えば理解しやすいですね。  
$x$: 質点の座標  
$\rho$ : 線密度   
の時、重心の座標は  
$$
x_G = \frac{\int x \rho (x) dx}{\int \rho (x) dx}
$$

---

## $SpectralSpread(X)$
$$
SpectralSpread = \sqrt{\frac{\int (f-\hat{f})^2|X(f)|df}{\int |X(f)|df}}
$$
ここで、$\hat{f}$ は SpectralCentroid である。  
先程出てきた確率密度 $p(f)$を使って表すと、
$$
SpectralSpread = \sqrt{\int (f - \hat{f})^2 p(f) df }
$$
となります。  
明らかに偏差の式になってますね。 
つまり、スペクトルの広がりを表しています。  

---

## $SpectralEntropy(X)$
EnergyEntropy でやったことと同じです。  
シグナルをスペクトルで置き換えればいいです。

---

## $SpectralFlux(X)$
$$
SpectralFlux := \int \left(\frac{|X(f)|}{\int{|X(f)|}df}-\frac{|X(f)_{prev}|}{\int{|X(f)_{prev}|}df}   \right) ^2df
$$

ここで、$X(f)_{prev}$ は一つ前の $X(f)$ の値である。
各周波数$f$の強度$X(f)$の差を見ていますので、変化が大きいほど大きな値になります。  
局所的な変化にも反応しますが、スペクトルが全体的に変化したとにに大きく機能します。

---

## $SpectralRollOff(X)$

$$
argmin \left \{s | \alpha \int{|X(f)|^2}df < \int^{s}_{- \infty}{|X(f)|^2}df  \right\}/f_{max}
$$

$\alpha$ のデフォルト値は、0.9 です。  
ロールオフは初めは解釈が難しいですが、じっくり数式を見れば理解できます。

{ } 内の不等号を満たすような最小の$\alpha$は何かを考えることがポイントです。  
不等号の左辺から見ていきます。全区間で積分しているので、スペクトルの面積を計算しています。  
それに $\alpha=0.9$がかかっているので、全面積の90%になります。   
不等号の右辺の$s$は、積分をどこまで行うのかを示しています。   
不等号の意味を考えると、面積の90%程度になるためには、どこまで積分すれば良いのか？  
という意味になります。  
もし、低い音がたくさん含まれる音であれば、スペクトルは、低周波側に大きな値を持つスペクトルとなります。  
その場合は、$s$ は小さな値をとります。  
逆に、高い音が多く含まれている音であれば、$s$は大きくなります。  